# PMF of DNA-DNA Interactions

#### References
1. Yoo, J. & Aksimentiev, A. Improved Parameterization of Amine-Carboxylate and Amine-Phosphate Interactions for Molecular Dynamics Simulations Using the CHARMM and AMBER Force Fields. Journal of Chemical Theory and Computation 12, 430-443 (2016). https://doi.org/10.1021/acs.jctc.5b00967

2. Yoo, J., Kim, H., Aksimentiev, A. & Ha, T. Direct evidence for sequence-dependent attraction between double-stranded DNA controlled by methylation. Nature Communications 7, 11045 (2016). https://doi.org/10.1038/ncomms11045

3. Kang, H. et al. Sequence-dependent DNA condensation as a driving force of DNA phase separation. Nucleic Acids Research 46, 9401-9413 (2018). https://doi.org/10.1093/nar/gky639

<img src="../img/dna_dna.png" width=800 />

### **Preparation of simulation with a pair of dsDNA molecules**
- Prepare PDB and ITP files for a PBC dsDNA molecule.  
    - You can create new DNA moecules under PBC using our web server (http://yoo.skku.edu/apps)  
    - Otherwise, you can reuse PDB and ITP files from previous simulations.  
        - For example, I copied the following files from the simulation of DNA-PCNA complex.  
            Structure file: `dna.pdb`  
            Topology file for the system: `topol.top`  
            Moleculetype files for DNA: `dna.itp` `dna.hbonds.itp`  `dna.pbc.itp`  
            MDP files: `grompp.mdp`  `equil.mdp` `mini.mdp`  
        2. Remove everything but DNA in `dna.pdb` and `topol.top` using text editor. 
        - Box information must be specified in `dna.pdb`. If it is not specified, define it using `gmx editconf`.  
            - My box information in `dna.pdb`:  
                ```bash
                CRYST1  115.000  115.000   68.000  90.00  90.00  60.00 P 1           1
                ```
            - We need to have a pair of DNA. We will duplicate the existing DNA.  
                - `gmx editconf -f dna.pdb -o dna2.pdb -translate 4 0 0`  
                - Add `dna2.pdb` to the end of `dna.pdb` using text editor.  
                - Edit `topol.top` to specify that we have TWO DNA molecules.  
                    ```bash
                    [ molecules ]
                    ; Compound        #mols
                    DNA                 2
                    ```
        - Solvate.  
            - `gmx solvate -cp dna.pdb -cs -o dna_water.pdb -p topol.top`  
                - `dna_water.pdb` contains DNA and water molecules.  
                - `topol.top`: updated to specify the number of water molecules.  
        - Add ions.  
            - Prepare TPR file for genion.  
            - `gmx grompp -f mini.mdp -c dna_water.pdb -maxwarn 40` 
            - `gmx genion -neutral -conc 0.15 -s topol.tpr -o dna_water_ions.pdb -p topol.top`  
        - Minimize  
            - `gmx grompp -f mini.mdp -c dna_water_ions.pdb  -maxwarn 40`  
            - `gmx mdrun -nt 4 -gpu_id 0`  
                - `confout.gro`  contains the final structure of the minimization.  
                - But, `confout.gro` may have broken DNA molecules due to PBC.  
                - We can make the molecules whole again by using `gmx trjconv`  
                    - `gmx trjconv -f confout.gro -o conf.pdb -pbc mol -ur compact`  
        - Equilibration under NVT. Equilibration = 안정화, 예열, 본격적으로 production 전 제 자리 잡도록 도와줌.   
            - In `nvt.mdp` file,  
                ```bash
                ; Temperature coupling  
                tcoupl                   = v-rescale
                ; pressure coupling     
                Pcoupl                   = no
                ```

            - `gmx grompp -c conf.pdb -f nvt.mdp -maxwarn 40`  
            - `gmx mdrun -s topol.tpr -nt 8 -gpu_id 0 -deffnm nvt`  
        - Equilibration under NPT.   
            - In npt.mdp file,  
                ```bash
                ; Temperature coupling  
                tcoupl                   = v-rescale
                ; pressure coupling     
                Pcoupl                   = c-rescale
                Pcoupltype               = semiisotropic
                ; Time constant (ps), compressibility (1/bar) and reference P (bar)
                tau-p                    = 1.0
                compressibility          = 4.5e-5 4.5e-5
                ref-p                    = 1      1
                ```
            - `gmx grompp -c conf.pdb -f nvt.mdp -maxwarn 40`  
            - `gmx mdrun -s topol.tpr -nt 8 -gpu_id 0 -deffnm npt`  
            - To see the box fluctuation in `npt.xtc`,  
                - `gmx traj -f npt.xtc -ob box.xvg`  

### **PMF (potential of mean forces) simulation using pull codes in Gromacs**
- We will start pull simulations using `npt.gro` as a starting structure.  
- We will measure mean forces at a range from inter-DNA distance d=20Å to 34Å with a spacing of 2 Å: 22, 24, 26, 28, 30, - 32, 34Å (7 windows in total).  
- Make a directory pull20 and `cd` to pull20, which is a window at DNA-DNA distance = 20Å.  
    - We will reuse `topol.top` and `npt.gro` from the equilibration.  
        `ln -s ../topol.top`  
        `ln -s ../dna.itp`  
        `ln -s ../dna.hbonds.itp`  
        `ln -s ../dna.pbc.itp`   
        `ln -s ../npt.gro`  
    - Make groups for DNA1 and DNA2 using `gmx make_ndx`  
        - `gmx make_ndx -f npt.gro`  
            ```bash
            ri 1-40
            ri 41-80
            name 9 DNA1
            name 10 DNA2
            q
            ```
        - Group file will be written to `index.ndx`.  
    - Copy `npt.mdp` to `pull.mdp`  
        - `cp ../npt.mdp pull.mdp`  
        - Edit `pull.mdp` using text editor to add pull codes.  
            ```bash
            ; COM PULLING          
            pull                     = yes
            ; Cylinder radius for dynamic reaction force groups (nm)
            pull-nstxout             = 50
            pull-nstfout             = 50
            ; Number of pull groups 
            pull-ngroups             = 2
            ; Number of pull coordinates
            pull-ncoords             = 1
            ; Group and coordinate parameters
            pull-group1-name         = DNA1 ; must be consistent with NDX file.
            pull-group2-name         = DNA2 ; must be consistent with NDX file.
            pull-coord1-type         = umbrella
            pull-coord1-geometry     = distance
            pull-coord1-groups       = 1 2
            pull-coord1-dim          = Y Y N    ; distance using deltaX and deltaY.
            pull-coord1-init         = 2.0  ; spring length in nm
            pull-coord1-k            = 1000 ; spring constant
            ```
- `grompp`  
    - `gmx grompp -f pull.mdp -maxwarn 40 -c npt.gro -n index.ndx`  
- `gmx mdrun -nt 8 -gpu_id 0`  
    - Outputs:  
        - `pullx.xvg`: spring length as a function of time.  
        - `pullf.xvg`: spring force as a function of time. (f = -kx)  

- Repeat the same pull simulation at d=22Å window.  
    - Copy (or rsync) everything from pull20 to pull22.  
    - Edit pull.mdp to modify spring length to 2.2 nm.  
        ```bash
        pull-coord1-init         = 2.2  ; spring length in nm
        ```
    - Then, `gmx grompp` & `gmx mdrun`  
- Repeat the same thing for remaining windows.  
- Plot mean forces as a function of spring length, d.  
- Integrate the mean force curve using pythong, origin, excel... => ∆G(d) = PMF.  
    - Otherwise, you can use gmx wham to get more accurate PMF.  
- Compare ∆G curves before and after subtracting the entropic contribution from the Jacobian factor: kT log(r).  
    - Note that the entropic contribution in water-water PMF was 2 kT log(r).  

<img src="../img/dnadna_ion.png" width=300 />